In [110]:
#Import libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')
import prince
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from collections import defaultdict
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode()
from kmodes.kmodes import KModes

In [111]:
#Load and view dataset
df = pd.read_csv('clean_invoice.csv')
pd.set_option('display.max_columns', 100)
df = df.dropna()
df.head(10)

Cust Type Customer No.     District Invoice Date  Invoice No Invoice Time  \
0    Retail        67849  Maharashtra   02-01-2015  7005200002     11:30:36   
1    Retail        84419  Maharashtra   03-01-2015  7005200003     10:07:32   
2    Retail        81055  Maharashtra   03-01-2015  7005200004     11:12:57   
3    Retail        84419  Maharashtra   03-01-2015  7005200005     11:40:44   
4    Retail        18980  Maharashtra   03-01-2015  7005200006     12:07:17   
5    Retail        84457  Maharashtra   03-01-2015  7005200007     12:10:08   
6    Retail        12523  Maharashtra   03-01-2015  7005200008     12:12:31   
7    Retail        55220  Maharashtra   03-01-2015  7005200009     12:18:54   
8    Retail        84511  Maharashtra   03-01-2015  7005200010     13:07:27   
9    Retail        84457  Maharashtra   03-01-2015  7005200011     15:54:21   

   Job Card No JobCard Date JobCard Time  KMs Reading  Labour Total  \
0       168303   13-12-2014     14:29:43        49317       1203.14   
1       173997   02-01-2015     14:12:18        78584        804.26   
2       173889   02-01-2015     11:40:44        33985        180.19   
3       174158   03-01-2015     10:12:32        78584          0.00   
4       173860   02-01-2015     10:45:42        50057       1202.97   
5       174053   02-01-2015     15:15:10        13122          0.00   
6       173953   02-01-2015     12:15:55        26964          0.00   
7       174254   03-01-2015     12:02:32        12943          0.00   
8       174162   03-01-2015     10:21:26        18563          0.00   
9       174203   03-01-2015     11:11:00         5443        698.92   

             Make  Misc Total        Model  OSL Total         Order Type  \
0  GENERAL MOTORS        0.00        SPARK     500.06       Paid Service   
1     TATA MOTORS      197.03       INDICA       0.00  SMC Value Package   
2   MARUTI SUZUKI        0.00          ZEN       0.00    Running Repairs   
3     TATA MOTORS        0.00       INDICA       0.00     SMC Redemption   
4         HYUNDAI      100.00  SANTRO XING     499.99       Paid Service   
5          TOYOTA        0.00       INNOVA     851.70    Running Repairs   
6           HONDA        0.00         CITY     749.42    Running Repairs   
7   MARUTI SUZUKI        0.00  SWIFT DZIRE       0.00    Running Repairs   
8     TATA MOTORS        0.00       INDIGO       0.00    Running Repairs   
9   MARUTI SUZUKI       49.99         ALTO     499.95       Paid Service   

   Parts Total  Recovrbl Exp   City    invoice_date_time    jobcard_date_time  \
0      2348.75           0.0  thane  2015-02-01 11:30:36  2014-12-13 14:29:43   
1         0.00           0.0  thane  2015-03-01 10:07:32  2015-02-01 14:12:18   
2        52.95           0.0  thane  2015-03-01 11:12:57  2015-02-01 11:40:44   
3         0.00           0.0  thane  2015-03-01 11:40:44  2015-03-01 10:12:32   
4       944.16           0.0  thane  2015-03-01 12:07:17  2015-02-01 10:45:42   
5        14.22           0.0  thane  2015-03-01 12:10:08  2015-02-01 15:15:10   
6         0.00           0.0  thane  2015-03-01 12:12:31  2015-02-01 12:15:55   
7         0.00           0.0  thane  2015-03-01 12:18:54  2015-03-01 12:02:32   
8         0.00           0.0  thane  2015-03-01 13:07:27  2015-03-01 10:21:26   
9      2663.75           0.0  thane  2015-03-01 15:54:21  2015-03-01 11:11:00   

                 service_time  service_time_hours  service_time_days    Total  \
0  49 days 21:00:53.000000000            1197.015               50.0  4051.95   
1  27 days 19:55:14.000000000             667.921               28.0  1001.29   
2  27 days 23:32:13.000000000             671.537               28.0   233.14   
3   0 days 01:28:12.000000000               1.470                0.0     0.00   
4  28 days 01:21:35.000000000             673.360               28.0  2747.12   
5  27 days 20:54:58.000000000             668.916               28.0   865.92   
6  27 days 23:56:36.000000000             671.943     

In [112]:
#Take subset of data for customer value segmentation
df_value = df[['District', 'Make', 'Order Type', 'km_bins', 'repair month']]

In [113]:
#Reduce Dimensionality by mapping values to general labels

#Dictionary for Make
make = {'GENERAL MOTORS' : 'mid-range', 'TATA MOTORS': 'budget', 'MARUTI SUZUKI': 'budget', 'HYUNDAI': 'mid-range',
       'TOYOTA': 'mid-range', 'HONDA': 'mid-range', 'FIAT': 'budget', 'FORD': 'mid-range', 'SKODA': 'high-end',
       'MAHINDRA &  MAHINDRA': 'budget', 'VOLKSWAGEN' : 'mid-range', 'NISSAN': 'mid-range', 'RENAULT': 'mid-range',
       'MERCEDES BENZ': 'high-end', 'MITSUBISHI MOTORS': 'mid-range', 'DAEWOO': 'budget', 'BMW': 'high-end',
       'FORCE': 'traveller', 'SONALIKA': 'traveller', 'VOLVO': 'high-end', 'HINDUSTAN MOTORS': 'traveller', 'AUDI':'high-end', 
       'PREMIER\xa0AUTOMOBILES': 'traveller', 'PORCHE':'high-end', 'LAND ROVER': 'high-end', 'JEEP': 'high-end',
       'MORRIS': 'budget', 'SAN MOTORS': 'mid-range'}

#Dictionary for DISTRICT
district = {'Maharashtra': 'Western', 'Gujarat': 'Western', 'Dadra and Nagar Hav.': 'Western',
            'Daman and Diu': 'Western', 'Rajasthan': 'Western', 'Goa' : 'Western',
           'Madhya Pradesh':'Central', 'Uttar Pradesh':'Central', 'Chhattisgarh': 'Central',
           'Karnataka':'Southern', 'Andhra Pradesh':'Southern', 'Telangana':'Southern', 'Tamil Nadu':'Southern',
            'Kerala':'Southern', 'Puducherry': 'Southern',
            'Tripura':'Eastern' ,'Manipur':'Eastern', 'Arunachal Pradesh':'Eastern', 'Megalaya':'Eastern',
            'Bihar':'Eastern', 'Jharkhand':'Eastern', 'West Bengal':'Eastern', 'Nagaland':'Eastern', 'Sikkim':'Eastern',
           'Odisha':'Eastern', 'Mizoram':'Eastern', 'Assam': 'Eastern',
           'Punjab':'Northern', 'Haryana':'Northern', 'Delhi':'Northern', 'Uttarakhand':'Northern',
            'Chandigarh':'Northern', 'Himachal Pradesh':'Northern', 'Jammu and Kashmir': 'Northern',
           'Lakshadweep':'Island', 'Andaman and Nico.In.':'Island'}

#Dictionary for km_bins
km_bins = {'0 - 10000':'0 - 50000', '10000 - 20000': '0 - 50000', '20000 - 50000': '0 - 50000',
          '50000 - 100000': '50000 - 200000', '100000 - 200000': '50000 - 200000',
          '200000 - 500000': '>200000', '>500000': '>200000'}

#Dictionary for months
seasons = {10:'Winter', 11:'Winter', 12:'Winter', 1:'Winter', 2:'Winter',3:'Summer', 4:'Summer', 
           5: 'Summer', 6:'Monsoon', 7:'Monsoon', 8:'Monsoon', 9: 'Monsoon'}

#Map district, make and km_bins to dictionaries
df_value['Make'] = df_value['Make'].map(make)
df_value['District'] = df_value['District'].map(district)
df_value['km_bins'] = df_value['km_bins'].map(km_bins)
df_value['repair month'] = df_value['repair month'].map(seasons)

In [114]:
#One Hot encode the features to be clustered
df_value2 = pd.get_dummies(df_value, drop_first=True)

In [ ]:
#Find optimal number of clusters
dist = []
for i in range(1, 11):
    km = KModes(n_clusters = i, init = 'Cao', max_iter = 300, n_init = 10, random_state = 42)
    km.fit(df_value2)

    dist.append(km.cost_)

In [ ]:
#Plot graph for clusters
plt.figure(figsize = (10, 10))
sns.lineplot(range(1,10), dist)
plt.title('Optimal number of clusters')
plt.xlabel('Number of Clusters')
plt.ylabel('Cost')
plt.show()

In [101]:
#Use 7 as number of clusters
km = KModes(n_clusters = 6, init = 'Cao', max_iter = 300, n_init = 10, random_state = 42)
km.fit(df_value2)
cluster = km.predict(df_value2)

In [106]:
#Add cluster column to df_value
df_value2['cluster'] = cluster

In [107]:
#View the attributes for each cluster
grouped4 = df_value2.groupby(['cluster']).agg(lambda x:x.value_counts().index[0])
grouped4